In [4]:
from clickhouse_driver import connect
from clickhouse_driver import Client
client = Client('localhost', settings={'use_numpy': True})
client.query_dataframe(
    'SELECT number AS x, (number + 100) AS y '
    'FROM system.numbers LIMIT 10000'
)

,x,y
0,0,100
1,1,101
2,2,102
3,3,103
4,4,104
...,...,...
9995,9995,10095
9996,9996,10096
9997,9997,10097
9998,9998,10098


In [7]:
# from clickhouse_driver.dbapi import connect
# conn = connect(database="default", user="default", password="default", 
#                host="localhost")


In [5]:
# Writing pandas DataFrame is also supported with insert_dataframe:

client = Client('localhost', settings={'use_numpy': True})
client.execute(
   'CREATE TABLE test_num (x Int64, y Int64) Engine = Memory'
)

df = client.query_dataframe(
    'SELECT number AS x, (number + 100) AS y '
    'FROM system.numbers LIMIT 10000'
)
client.insert_dataframe('INSERT INTO test_num VALUES', df)

10000

In [10]:
import pandas as pd

client = Client('localhost', settings={'use_numpy': True})
client.execute(
   'CREATE TABLE if not exists test_x ('
   'a Nullable(Int64),'
   'b Nullable(Float64),'
   'c Nullable(String)'
   ') Engine = Memory'
)

df = pd.DataFrame({
    'a': [1, None, None],
    'b': [1.0, None, np.nan],
    'c': ['a', None, np.nan],
}, dtype=object)
client.insert_dataframe('INSERT INTO test_x VALUES', df)

client.query_dataframe('SELECT * FROM test_x')

,a,b,c
0,1,1,a
1,None,None,None
2,None,NaN,None


In [11]:
# It’s important to specify dtype during dataframe creation:

good_df = pd.DataFrame({
    'a': [1, None, None],
    'b': [1.0, None, np.nan],
    'c': ['a', None, np.nan],
}, dtype=object)
good_df

,a,b,c
0,1,1,a
1,None,None,None
2,None,NaN,NaN


In [2]:
client.query_dataframe('''SELECT
    title,
    length(NER),
    length(directions)
FROM recipes
WHERE has(NER, 'strawberry')
ORDER BY length(directions) DESC
LIMIT 10''')

,title,length_NER_,length_directions_
0,Chocolate-Strawberry-Orange Wedding Cake,24,126
1,Strawberry Cream Cheese Crumble Tart,19,47
2,Charlotte-Style Ice Cream,11,45
3,Sinfully Good a Million Layers Chocolate Layer...,31,45
4,Sweetened Berries With Elderflower Sherbet,24,44
5,Chocolate-Strawberry Mousse Cake,15,42
6,Rhubarb Charlotte with Strawberries and Rum,20,42
7,Old-Fashioned Ice Cream Sundae Cake,17,37
8,Chef Joey's Strawberry Vanilla Tart,7,37
9,Watermelon Cake,16,36


In [3]:
sql='''SELECT arrayJoin(directions)
FROM recipes
WHERE title = 'Chocolate-Strawberry-Orange Wedding Cake'
'''
client.query_dataframe(sql)

,arrayJoin_directions_
0,Position 1 rack in center and 1 rack in bottom...
1,Butter one 5-inch-diameter cake pan with 2-inc...
2,Dust pans with flour; line bottoms with parchm...
3,Combine 1/3 cup orange juice and 2 ounces unsw...
4,Stir mixture over medium-low heat until chocol...
...,...
121,Remove top and middle cake tiers.
122,Remove dowels from cakes.
123,Cut top and middle cakes into slices.
124,To cut 12-inch cake: Starting 3 inches inward ...
